In [153]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from ipynb.fs.full.UseCase4_PredictTravelInsuranceClaim_FE import *

In [154]:
file_path=r'C:\Users\ADMIN\Desktop\UseCase3_EDAOutput.xlsx'
col_list=['age','sex','bmi','steps','children','smoker','region','charges','insuranceclaim']

In [155]:
#Read the file
customer_data=pd.read_excel(file_path)
customer_data.head()

,age,sex,bmi,steps,children,smoker,region,charges,insuranceclaim,charges_disc,age_disc,bmi_disc,steps_disc
0,19,0,27.900,3009,0,1,3,16884.92400,1,"(13651.59, 19916.45]","(17.999, 22.6]","(27.12, 30.84]","(2999.999, 3701.0]"
1,18,1,33.770,3008,1,0,2,1725.55230,1,"(1121.869, 7386.73]","(17.999, 22.6]","(30.84, 34.56]","(2999.999, 3701.0]"
2,28,1,33.000,3009,3,0,2,4449.46200,0,"(1121.869, 7386.73]","(27.2, 31.8]","(30.84, 34.56]","(2999.999, 3701.0]"
3,33,1,22.705,10009,0,0,1,21984.47061,0,"(19916.45, 26181.31]","(31.8, 36.4]","(19.68, 23.4]","(9309.0, 10010.0]"
4,32,1,28.880,8010,0,0,1,3866.85520,1,"(1121.869, 7386.73]","(31.8, 36.4]","(27.12, 30.84]","(7907.0, 8608.0]"


In [258]:
#Adding steps and children increases the VIF, so using only 3 features in the model.
X=customer_data[['bmi','smoker','children','charges']]
y = customer_data.insuranceclaim
X.head()

,bmi,smoker,children,charges
0,27.900,1,0,16884.92400
1,33.770,0,1,1725.55230
2,33.000,0,3,4449.46200
3,22.705,0,0,21984.47061
4,28.880,0,0,3866.85520


In [259]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])] ## Replace X with your indepedent data type
vif_s =pd.Series(vif, index =X.columns)
print(vif_s)

bmi         3.358249
smoker      3.480352
children    1.791463
charges     6.425436
dtype: float64


In [260]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [261]:
#Lets do outlier handing for children column as children 4 and 5 category is around 1.5%
(customer_data[customer_data.children==3].count()/len(customer_data)*100)[0]
rare_freq_imputation(X_train,X_test,'children')
X_train.head()

C:\Users\ADMIN\UseCase4_PredictTravelInsuranceClaim_FE.ipynb:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "</style>\n",
C:\Users\ADMIN\UseCase4_PredictTravelInsuranceClaim_FE.ipynb:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "<table border=\"1\" class=\"dataframe\">\n",


,bmi,smoker,children,charges,children_freq_imp
621,34.100,1,4,40182.24600,0
194,34.430,0,0,1137.46970,0
240,36.670,1,2,38511.62830,2
1168,35.200,0,2,4670.64000,2
1192,32.395,0,1,13019.16105,1


In [262]:
X_train.drop('children',axis=1,inplace=True)
X_test.drop('children',axis=1,inplace=True)

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [263]:
classifier=LogisticRegression()

In [264]:
#Before calling the logistic regressor lets do the standard scaling
from sklearn.preprocessing import StandardScaler
X_train_m1=X_train
X_test_m1=X_test
y_train_m1=y_train
y_test_m1=y_test

In [265]:
print(col_list[2],col_list[3],col_list[4],col_list[5],col_list[7])

bmi steps children smoker charges


In [266]:
# set up the scaler
scaler = StandardScaler()

# fit and transform train and test sets
X_train_scaled=scaler.fit_transform(X_train_m1,[col_list[2],'children_freq_imp',col_list[5]])
X_test_scaled=scaler.fit_transform(X_test_m1,[col_list[2],'children_frq_imp',col_list[5]])


In [267]:
# let's transform the returned NumPy arrays to dataframes 

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [268]:
import numpy as np
np.round(X_train.describe(), 1)

,bmi,smoker,charges,children_freq_imp
count,1070.0,1070.0,1070.0,1070.0
mean,30.7,0.2,13201.2,0.9
std,6.2,0.4,11979.2,1.0
min,16.0,0.0,1121.9,0.0
25%,26.3,0.0,4746.5,0.0
50%,30.4,0.0,9440.1,1.0
75%,34.8,0.0,16584.3,2.0
max,53.1,1.0,63770.4,3.0


In [269]:
np.round(X_train_scaled.describe(), 1)

,bmi,smoker,charges,children_freq_imp
count,1070.0,1070.0,1070.0,1070.0
mean,-0.0,0.0,-0.0,0.0
std,1.0,1.0,1.0,1.0
min,-2.4,-0.5,-1.0,-0.9
25%,-0.7,-0.5,-0.7,-0.9
50%,-0.1,-0.5,-0.3,0.1
75%,0.7,-0.5,0.3,1.0
max,3.6,2.0,4.2,2.0


In [270]:
X_train_scaled.head()

,bmi,smoker,charges,children_freq_imp
0,0.545305,1.971255,2.253387,-0.891975
1,0.598672,-0.507291,-1.007529,-0.891975
2,0.960921,1.971255,2.113861,1.028374
3,0.723195,-0.507291,-0.712448,1.028374
4,0.269575,-0.507291,-0.015202,0.068199


In [271]:
X_train.head()

,bmi,smoker,charges,children_freq_imp
621,34.100,1,40182.24600,0
194,34.430,0,1137.46970,0
240,36.670,1,38511.62830,2
1168,35.200,0,4670.64000,2
1192,32.395,0,13019.16105,1


In [272]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression()

In [273]:
y_pred=classifier.predict(X_test_scaled)

In [274]:
print(y_pred)
print("\n")
print(y_test)

[1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0
 1 0 1 1 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0
 1 1 0 1 0 1 0 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0
 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 1 1 1
 0 0 1 1 0 0 1 1 0]


578     1
610     0
569     1
1034    1
198     0
981     0
31      1
1256    0
1219    0
1320    0
613     0
1107    1
1263    0
406     0
795     1
970     1
824     0
141     1
1173    0
1042    1
966     1
467     0
1098    1
757     1
1097    1
319     0
1286    1
459     0
5       0
517     0
54      0
1062    1
722     1
45      1
989     1
1032    1
1121    1
34      1
667     1
1268    1
863     0
108     1
638     1
55      1
161     1
101

In [275]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score,log_loss
accuracy_score(y_test,y_pred)

0.8843283582089553

In [276]:
print(confusion_matrix(y_test,y_pred))
log_loss(y_test,y_pred)

[[103  20]
 [ 11 134]]


3.995216642503116

In [277]:
precision_score(y_test,y_pred)

0.8701298701298701

In [278]:
recall_score(y_test,y_pred)

0.9241379310344827

In [279]:
f1_score(y_test,y_pred)

0.8963210702341137

In [280]:
#Lets check the output with cross validation.
from sklearn.model_selection import cross_val_score 
classifier_avg = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 5,scoring='f1') 
print (classifier_avg.mean()*100)
print (classifier_avg.std()*100)

85.89181801320258
2.950926445628889


In [281]:
import sklearn.metrics
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [282]:
customer_data[customer_data.children==0].count()/len(customer_data)*100

age               42.899851
sex               42.899851
bmi               42.899851
steps             42.899851
children          42.899851
smoker            42.899851
region            42.899851
charges           42.899851
insuranceclaim    42.899851
charges_disc      42.899851
age_disc          42.899851
bmi_disc          42.899851
steps_disc        42.899851
dtype: float64